In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/baml_contest

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/baml_contest


In [2]:
%pip install sentencepiece

In [3]:
# %pip install transformers

In [4]:
import pandas as pd
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
from tqdm.auto import tqdm, trange
import os
from sklearn.model_selection import train_test_split

In [5]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")
set_seed()

Random seed set as 42


In [6]:
df = pd.read_excel('../dataset_making/df_result_final.xlsx')
df = df[df.answer!='-']
df.columns = ['X', 'Y']
df = df[~df['X'].isna()]
df = df[~df['Y'].isna()]
print(df.shape[0])
df.head()

9906


,X,Y
0,"Хорошая Лерочка ты женщина, хозяюшка и заботл...",Текст представляет собой набор комментариев по...
1,Лапкозаврик встраивает узел в пустую ячейку и ...,"В посте обсуждаются различные аспекты игры ""Ла..."
2,интересно. эмпирическим методом надо это прове...,Комментарии обсуждают рацион правильного питан...
3,"Юрий хороший вы человек, да поможет вам Бог .❤...","В этом комментарии обсуждается, что ролики Юри..."
4,Вау... прекрасно ❤ Как всегда идеально❤ Атмосф...,В комментариях к посту автор выражает свою рад...


In [7]:
repl = pd.read_excel('../src/replacements_long.xlsx', header=None)
repl.columns = ['r']
repl = list(repl.r.values)

In [8]:
def repls(x):

    if x[0]=='"' and x[-1]=='"':
        x = x[1:-1]

    very_pop_intro = 'Комментарии варьируются от положительных до отрицательных. '
    if very_pop_intro in x[0:len(very_pop_intro)] and len(x)*2 > len(very_pop_intro):
        x = x.replace(very_pop_intro, '')

    for _ in range(10):
        for el in repl:
            if el in x[0:len(el)]:
                x = x.replace(el, '')
    return x.capitalize()


df['Y'] = df['Y'].apply(lambda x: repls(x))

In [9]:
df['X'] = df['X'].apply(lambda x: x[0:1000])
df = df[df['Y']!='']
df.shape[0]

9906

In [10]:
df_train, df_test = train_test_split(df.dropna(), test_size=0.15, random_state=1)
pairs = df_train[['X', 'Y']].values.tolist()

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

raw_model = 'd0rj/rut5-base-summ'
model = T5ForConditionalGeneration.from_pretrained(raw_model).cuda()
tokenizer = T5Tokenizer.from_pretrained(raw_model)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

# new_model_name = '../src/my_custom_model_4'
# model = T5ForConditionalGeneration.from_pretrained(new_model_name, local_files_only=True).cuda()
# # model = T5ForConditionalGeneration.from_pretrained(new_model_name, local_files_only=True)
# tokenizer = T5Tokenizer.from_pretrained(new_model_name, local_files_only=True)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
batch_size = 4 # 8
report_steps = 50
epochs = 5

# batch_size = 8
# report_steps = 5
# epochs = 1

In [23]:
# pairs = pairs[0:20]

In [24]:
model.train()
losses = []
best_test_loss = 9999999
breaker = 0
stop_breaker = 100
for epoch in range(epochs):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        model.train()
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())

        if i % report_steps == 0:

            model.eval()
            losses_test = []
            pairs_test = df_test[['X', 'Y']].values.tolist()
            random.shuffle(pairs_test)
            for j in range(0, int(len(pairs_test) / batch_size)):
                batch = pairs_test[j * batch_size: (j + 1) * batch_size]
                x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
                y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
                y.input_ids[y.input_ids == 0] = -100
                loss = model(
                    input_ids=x.input_ids,
                    attention_mask=x.attention_mask,
                    labels=y.input_ids,
                    decoder_attention_mask=y.attention_mask,
                    return_dict=True
                ).loss
                losses_test.append(loss.item())

            print('step', i, 'loss:', np.mean(losses[-report_steps:]), 'test_loss:', np.mean(losses_test))

            test_loss = np.mean(losses_test)
            if test_loss < best_test_loss:
                best_test_loss = test_loss
                print('loss теста снизился, сохраняем модель')
                new_model_name = 'src/my_custom_model_4'
                model.save_pretrained(new_model_name)
                tokenizer.save_pretrained(new_model_name)
            else:
                breaker += 1

            if breaker == stop_breaker:
                raise Exception('Заканчиваем обучение')

EPOCH 0


  0%|          | 0/2105 [00:00<?, ?it/s]

step 0 loss: 2.777674674987793 test_loss: 2.251300893703882
loss теста снизился, сохраняем модель
step 50 loss: 2.595400447845459 test_loss: 2.2488900426263116
loss теста снизился, сохраняем модель
step 100 loss: 2.551977863311768 test_loss: 2.2440938628266123
loss теста снизился, сохраняем модель
step 150 loss: 2.5316172289848327 test_loss: 2.250102803713549
step 200 loss: 2.4950809979438784 test_loss: 2.250645996425351
step 250 loss: 2.562086486816406 test_loss: 2.249828809676466
step 300 loss: 2.533814187049866 test_loss: 2.2497527541497324
step 350 loss: 2.5412562251091004 test_loss: 2.24868938222407


OutOfMemoryError: ignored

In [13]:
model.eval()

def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device)
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [14]:
sample = df_train.sample(50, random_state=42)
for i, row in sample.iterrows():
    print(row.X)
    print('real:', row.Y)
    print('model: ', repls(answer(row.X)))
    print('---')

Ядогадываюсь Интересно, интересно😊
Буду рада пересечься) Это та феечка Винкс что ли? Да уже всё понятно по этому коллажу, пхпх Но спойлерить не буду, на всякий 🤫 А почему все решили спойлернуть коллажами из пинтереста 😐🤨 не уж-то тетя из группы татту?) это леди?????
real: Коллажи из пинтереста, которые могут быть спойлером к новому сезону сериала "винкс". некоторые пользователи предполагают, что один из коллажей может быть связан с персонажем из оригинального сериала.
model:  Пользователи обсуждают коллаж, который они хотели бы спойлерить, и высказывают свое мнение о том, как это было сделано.
---
А про Пиноккио оставят%? <img src="https://cdn4.cdn-telegram.org/file/FSFzmrYMMH6qkIBefN961jOEB3Ukq_a0tdcXro6cjX2ujHwfSgpTMgEVbUSQNLxAc5hb7MsG2ZVHtzjf8VKLDY7ZnYcnwrS7cVyPFjvcCcbYGAAquKs6DaeNGG2OWjYnYtYuQmjNESTKLzAgkbiLx6Hu8ZYR3zSuwQWu3sEFpaSwcU2zaZCIeb5KUoyHwLX5G9hXFqBZgJTbvadMc9Jv4LcjeJz7NcO0VyyoMm1sNOHvB1TMZ8a0ohhWRz6Gbn3tNqAMvC7vIrQPsiysI5SUr5xj0IXxu-5SsXOVn8r_gVfnX4XmOKnQvLTO_xyEffwjVNpDs

In [15]:
sample = df_test.sample(50, random_state=42)
for i, row in sample.iterrows():
    print(row.X)
    print('real:', row.Y)
    print('model: ', repls(answer(row.X)))
    print('---')

Парни, пишите мне в личку, и я вышлю вам свои фото абсолютно бесплатно. Жду тебя солнышко. Знаем, очень хочется высказаться, но сначала прочитайте <a href="https://telegra.ph/Pravila-kommentariev-i-chata-Super-03-07">правила чата</a> Super 😉
Правила комментариев и чата Super
Запрещено: — Любые виды рекламы (в том числе в описании аккаунта) и попрошайничество; — Порнография и ссылки на нее; — Нецензурные высказывания; — Ссылки на сторонние сайты и телеграм-каналы;  — Любые виды шок-контента: трупы, насилие и тд; — Оскорбления участников чата и любых лиц (по расовому, гендерному признаку и т.д);  — Cрачи на политические и религиозные темы. Правила: — Данный чат ведется на русском языке; — Модераторы всегда правы, действия модерации не обсуждаются. Модерация не допускает разжигания… 💥 Удaлeннaя paбoтa для подрocткoв 14+, в биo 👀 👍👍 Супер наймите редактора который будет банить про--- к и скам ) Мда.... И зачем нам эта не интересная информация???? Только Ким не дает толком бывшему с детьми 

Сохранение предиктов по трейну и тесту в файлы, чтобы потом выбрать вводные конструкции

In [ ]:
answers = []
sample = df_test.copy()
for i, row in sample.iterrows():
    # print(repls(answer(row.X)))
    answers.append(repls(answer(row.X)))
    if i % 100==0:
      print(i)


3200
4200
100
1100
8800
2000
7100
5500
1000
3400
1300
2700
7300
1800
3500
5800
800


In [ ]:
df[['Y']].to_excel('clean_df.xlsx', index=False)

In [ ]:
# sample = df_train.sample(50, random_state=42)
# for i, row in sample.iterrows():
#     print(row.X)
#     print('real:', row.Y)
#     print('model: ', repls(repls(answer(row.X))))
#     print('---')

import math
N = df_train.shape[0]
BATCH_SIZE = 4
steps = math.ceil(N / BATCH_SIZE)
result = []
SEQ_LEN = 1500

for i in range(steps):
    if i != steps:
        batch = list(df_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE]['X'].values)
    else:
        batch = list(df_train[i*BATCH_SIZE:]['X'].values)

    input_ids = tokenizer(batch, return_tensors='pt', truncation=True,
                          max_length=SEQ_LEN, padding=True).input_ids.to(device)
    outputs = model.generate(input_ids)
    for el in outputs:
        summary = repls(tokenizer.decode(el, skip_special_tokens=True))
        result.append(summary)

    if i % 500 == 0:
        print(i)


0
500
1000
1500
2000


KeyboardInterrupt: ignored

In [ ]:
with open('result.txt', 'w') as fp:
    for item in result:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')


# with open('answers.txt', 'w') as fp:
#     for item in answers:
#         # write each item on a new line
#         fp.write("%s\n" % item)
#     print('Done')

Done


In [ ]:
# new_model_name = 'my_custom_model'
# model.save_pretrained(new_model_name)
# tokenizer.save_pretrained(new_model_name)

In [ ]:
breaker

30

In [ ]:
100 - 13 - 46 - 41

0